In [1]:
!pip install sklearn
!pip install boto3
!pip install pandas

In [2]:
import pandas as pd
import sklearn
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn import metrics
import boto3
from sklearn import svm
from sklearn import datasets
import pickle
from joblib import dump, load

In [3]:
s3_client = boto3.client('s3')

In [4]:
!wget -c https://bucket-turismo.s3.amazonaws.com/TURISMO_CONSOLIDADO.csv

--2023-12-07 01:03:59--  https://bucket-turismo.s3.amazonaws.com/TURISMO_CONSOLIDADO.csv
Resolving bucket-turismo.s3.amazonaws.com (bucket-turismo.s3.amazonaws.com)... 54.231.163.57, 54.231.224.153, 3.5.6.179, ...
Connecting to bucket-turismo.s3.amazonaws.com (bucket-turismo.s3.amazonaws.com)|54.231.163.57|:443... connected.
HTTP request sent, awaiting response... 403 Forbidden
2023-12-07 01:03:59 ERROR 403: Forbidden.



In [5]:
response = s3_client.list_objects(Bucket="bucket-turismo")
response

{'ResponseMetadata': {'RequestId': '73JHBQJHZVA9CW3R',
  'HostId': 'kIG4+mAgq4JK+J/SvxdFngZtaGkXGkPhq5u6JULsPUCb9TjPhXbDAqS0VwrFIyyFfp9BRuDBkLI=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'kIG4+mAgq4JK+J/SvxdFngZtaGkXGkPhq5u6JULsPUCb9TjPhXbDAqS0VwrFIyyFfp9BRuDBkLI=',
   'x-amz-request-id': '73JHBQJHZVA9CW3R',
   'date': 'Thu, 07 Dec 2023 01:04:01 GMT',
   'x-amz-bucket-region': 'us-east-1',
   'content-type': 'application/xml',
   'transfer-encoding': 'chunked',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'IsTruncated': False,
 'Marker': '',
 'Contents': [{'Key': 'TURISMO_CONSOLIDADO.csv',
   'LastModified': datetime.datetime(2023, 12, 7, 0, 6, 19, tzinfo=tzlocal()),
   'ETag': '"2d890b12abd6b08c598646f098c8b5fb-2"',
   'Size': 34323328,
   'StorageClass': 'STANDARD',
   'Owner': {'DisplayName': 'giovanni.carignato',
    'ID': '6b86d16c4f4a613ff6386a18db113e8b732a281b580199d1b22186707900537a'}}],
 'Name': 'bucket-turismo',
 'Prefix': '',
 'MaxKeys': 1000,
 'Encodi

In [6]:
s3_client.download_file('bucket-turismo','TURISMO_CONSOLIDADO.csv','TURISMO_CONSOLIDADO.csv')

In [7]:
turismo_data = pd.read_csv('TURISMO_CONSOLIDADO.csv', delimiter=';', on_bad_lines='skip')

In [8]:
turismo_data.describe

<bound method NDFrame.describe of                          continente  cod_continente                      pais  \
0          AMERICA CENTRAL E CARIBE               2                      CUBA   
1          AMERICA CENTRAL E CARIBE               2                      CUBA   
2          AMERICA CENTRAL E CARIBE               2                      CUBA   
3          AMERICA CENTRAL E CARIBE               2                      CUBA   
4          AMERICA CENTRAL E CARIBE               2                      CUBA   
...                             ...             ...                       ...   
505063  CONTINENTE NAO ESPECIFICADO               8  PAISES NAO ESPECIFICADOS   
505064  CONTINENTE NAO ESPECIFICADO               8  PAISES NAO ESPECIFICADOS   
505065  CONTINENTE NAO ESPECIFICADO               8  PAISES NAO ESPECIFICADOS   
505066  CONTINENTE NAO ESPECIFICADO               8  PAISES NAO ESPECIFICADOS   
505067  CONTINENTE NAO ESPECIFICADO               8  PAISES NAO ESPECIFICAD

In [9]:
turismo_data.drop(['continente', 'cod_continente', 'pais', 'uf','via', 'cod_via', 'mes'], axis = 1, inplace = True)

In [10]:
turismo_data.isnull().sum()

cod_pais       0
cod_uf         0
ano            0
cod_mes        0
chegadas    2688
dtype: int64

In [11]:
turismo_data.dropna(subset=['chegadas'], inplace=True)

In [12]:
turismo_data.isnull().sum()

cod_pais    0
cod_uf      0
ano         0
cod_mes     0
chegadas    0
dtype: int64

In [13]:
turismo_data.describe

<bound method NDFrame.describe of         cod_pais  cod_uf   ano  cod_mes  chegadas
0             55       4  2012       10       0.0
1             55       4  2012        1       0.0
2             55       4  2012        9      10.0
3             55       4  2012        4      13.0
4             55       4  2012       12      17.0
...          ...     ...   ...      ...       ...
505063       999      99  2020        6       0.0
505064       999      99  2020        8       0.0
505065       999      99  2020        7       0.0
505066       999      99  2020        4       0.0
505067       999      99  2020        4       0.0

[502380 rows x 5 columns]>

In [14]:
re_train, re_test = train_test_split(turismo_data, train_size=0.7)
X_train = re_train.iloc[:,[0, 1, 2, 3]]
y_train = re_train.iloc[:,4]

X_test = re_test.iloc[:, [0, 1, 2, 3]]
y_test = re_test.iloc[:,4]

In [15]:
X_train.head()

,cod_pais,cod_uf,ano,cod_mes
202231,26,24,2014,6
357853,998,21,2021,7
99897,174,3,2018,8
298312,173,99,2017,4
51118,998,19,2019,3


In [16]:
y_train.head()

202231      0.0
357853      0.0
99897       0.0
298312      0.0
51118     399.0
Name: chegadas, dtype: float64

In [17]:
X_test.head()

,cod_pais,cod_uf,ano,cod_mes
421136,63,12,2022,11
318742,228,16,2015,11
70802,179,99,2019,12
394733,94,5,2021,9
246205,26,21,2016,1


In [18]:
y_test.head()

421136    0.0
318742    1.0
70802     0.0
394733    1.0
246205    0.0
Name: chegadas, dtype: float64

In [19]:
reg = linear_model.LinearRegression()

In [20]:
reg.fit(X=X_train,y=y_train)

LinearRegression()

In [21]:
reg.coef_

array([ -0.22365098,  -0.40632202, -20.55421983,  -7.06454749])

In [22]:
reg.intercept_

41692.97741640306

In [23]:
y_pred = reg.predict(X_test)

In [24]:
print('MSE',metrics.mean_squared_error(y_test,y_pred))

MSE 2275353.9009008054


In [25]:
s = pickle.dumps(reg)
reg2 = pickle.loads(s)
reg2.predict([[68,25,2025,12]])

/home/ubuntu/miniconda3/envs/jupyter/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


array([-39.45863289])

In [26]:
dump(reg, 'modelo_turismo.joblib')

['modelo_turismo.joblib']

In [27]:
s3_client.upload_file('modelo_turismo.joblib','bucket-turismo','modelo_turismo.joblib')